# Chap03 索引

In [1]:
import numpy as np
import pandas as pd

## 索引器
### **表**的列索引
1. 取出单列：通过`df[列名]`，返回值为Series
   - 如果列名不包含空格，也可以通过`df.列名`取出
2. 取出多列：通过`df[多个列名组成的列表]`，返回值为DataFrame
### **序列**的行索引
1. 取出单个索引的对应元素：`s[item]`，如果只有一个值，返回标量，如果有多个值，返回一个Series
2. 取出多个索引的对应元素：`s[items的列表]`
3. 切片`s[n1:n2:n3]`，如果前后端点的值重复出现，需要经过排序`sort_index`才能使用切片
### loc索引器
1. 基于元素的loc索引器：`loc[*,*]`
   - 第一个\*代表行的选择，第二个\*代表列的选择。如果第二个\*省略，则只筛选行
   - \*的位置有五类合法的对象：单个元素、元素列表、元素切片、布尔列表以及函数
   - 这里的函数必须以前面的四种合法形式之一为返回值，并且函数的输入值为DataFrame本身；也支持**lambda**表达式
   - 函数无法返回如start:end:step的切片形式，返回切片时要用slice对象进行包装
   - Series也可以使用loc索引
2. 基于位置的iloc索引器：`iloc[*,*]`
   - iloc的使用与loc完全类似，只不过是针对位置进行筛选
   - \*的位置有五类合法对象：整数、整数列表、整数切片、布尔列表以及函数
   -  Series也可以使用iloc索引
### query方法
1. 支持把字符串形式的查询表达式传入query方法
2. 其表达式的执行结果必须返回**布尔列表**
3. 所有属于Series的方法都可以被调用
4. query中引用带有空格的列名：使用`col name`的方式进行引用
5. 对于query中的字符串，如果要引用外部变量，只需在变量名前加`@`符号
### 随机抽样
1. `sample`函数
   - `n`抽样数量
   - `axis`抽样的方向（0为行、1为列）
   - `frac`抽样比例
   - `replace`是否放回
   - `weights`每个样本的抽样相对概率
### **注意**
1. 不要使用链式赋值，在对表或者序列赋值时，应当在使用一层索引器后直接进行赋值操作，这样做是由于进行多次索引后赋值是赋在临时返回的copy副本上，而没有真正修改元素
   ```python
   df_chain = pd.DataFrame([[0,0],[1,0],[-1,0]], columns = list('AB'))
   df_chain[df_chain.A!=0].B=1
   # 使用方括号列索引后，再使用点的列索引
   ```

In [2]:
df = pd.read_csv('./data/learn_pandas.csv', usecols = ['School', 'Grade', 'Name', 'Gender', 'Weight', 'Transfer'])
df.head()

,School,Grade,Name,Gender,Weight,Transfer
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,46.0,N
1,Peking University,Freshman,Changqiang You,Male,70.0,N
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,89.0,N
3,Fudan University,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Fudan University,Sophomore,Gaojuan You,Male,74.0,N


In [3]:
df_demo = df.set_index('Name')
df_demo.head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaopeng Yang,Shanghai Jiao Tong University,Freshman,Female,46.0,N
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaojuan Sun,Fudan University,Sophomore,Female,41.0,N
Gaojuan You,Fudan University,Sophomore,Male,74.0,N


In [7]:
df_demo.loc[df_demo.Weight>70,'School'].head()

Name
Mei Sun          Shanghai Jiao Tong University
Gaojuan You                   Fudan University
Xiaopeng Zhou    Shanghai Jiao Tong University
Xiaofeng Sun               Tsinghua University
Qiang Zheng      Shanghai Jiao Tong University
Name: School, dtype: object

In [8]:
df_demo.loc[df_demo.Grade.isin(['Freshman','Senior'])].head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaopeng Yang,Shanghai Jiao Tong University,Freshman,Female,46.0,N
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaoli Qian,Tsinghua University,Freshman,Female,51.0,N
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female,52.0,N


In [11]:
# 复合条件：|或、&且、~取反
condition = df.dtypes=='object'
df.loc[:,~condition]

,Weight
0,46.0
1,70.0
2,89.0
3,41.0
4,74.0
...,...
195,46.0
196,50.0
197,45.0
198,71.0


In [12]:
# *为函数的例子
def condition(x):
    condition_1_1 = x.School == 'Fudan University'
    condition_1_2 = x.Grade == 'Senior'
    condition_1_3 = x.Weight > 70
    condition_1 = condition_1_1 & condition_1_2 & condition_1_3
    condition_2_1 = x.School == 'Peking University'
    condition_2_2 = x.Grade == 'Senior'
    condition_2_3 = x.Weight > 80
    condition_2 = condition_2_1 & (~condition_2_2) & condition_2_3
    result = condition_1 | condition_2
    return result
df_demo.loc[condition]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Qiang Han,Peking University,Freshman,Male,87.0,N
Chengpeng Zhou,Fudan University,Senior,Male,81.0,N
Changpeng Zhao,Peking University,Freshman,Male,83.0,N
Chengpeng Qian,Fudan University,Senior,Male,73.0,Y


In [13]:
df_demo.loc[lambda x:'Quan Zhao', lambda x:'Gender']

'Female'

In [14]:
df_demo.loc[lambda x: slice('Gaojuan You', 'Gaoqiang Qian')]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaojuan You,Fudan University,Sophomore,Male,74.0,N
Xiaoli Qian,Tsinghua University,Freshman,Female,51.0,N
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female,52.0,N
Gaoqiang Qian,Tsinghua University,Junior,Female,50.0,N


In [15]:
df_demo.iloc[1,1]

'Freshman'

In [16]:
df_demo.iloc[[0,1],[0,1]]

,School,Grade
Name,,
Gaopeng Yang,Shanghai Jiao Tong University,Freshman
Changqiang You,Peking University,Freshman


In [17]:
df_demo.iloc[1:4, 2:4]

,Gender,Weight
Name,,
Changqiang You,Male,70.0
Mei Sun,Male,89.0
Xiaojuan Sun,Female,41.0


In [18]:
df_demo.iloc[lambda x: slice(1,4)]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaojuan Sun,Fudan University,Sophomore,Female,41.0,N


In [22]:
# 在使用布尔列表的时候要注意，不能传入Series而必须传入序列的Values，因此在使用布尔筛选的时候还是应当优先考虑loc
df_demo.iloc[(df_demo.Weight>80).values].head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Qiang Zheng,Shanghai Jiao Tong University,Senior,Male,87.0,N
Qiang Han,Peking University,Freshman,Male,87.0,N
Chengpeng Zhou,Fudan University,Senior,Male,81.0,N
Feng Han,Shanghai Jiao Tong University,Sophomore,Male,82.0,N


In [5]:
df.query('((School == "Fudan University")&'
         ' (Grade == "Senior")&'
         ' (Weight > 70))|'
         '((School == "Peking University")&'
         ' (Grade != "Senior")&'
         ' (Weight > 80))')

,School,Grade,Name,Gender,Weight,Transfer
38,Peking University,Freshman,Qiang Han,Male,87.0,N
66,Fudan University,Senior,Chengpeng Zhou,Male,81.0,N
99,Peking University,Freshman,Changpeng Zhao,Male,83.0,N
131,Fudan University,Senior,Chengpeng Qian,Male,73.0,Y


In [7]:
df.query('Weight > Weight.mean()').head()

,School,Grade,Name,Gender,Weight,Transfer
1,Peking University,Freshman,Changqiang You,Male,70.0,N
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,89.0,N
4,Fudan University,Sophomore,Gaojuan You,Male,74.0,N
10,Shanghai Jiao Tong University,Freshman,Xiaopeng Zhou,Male,74.0,N
14,Tsinghua University,Senior,Xiaomei Zhou,Female,57.0,N


In [8]:
df.head()

,School,Grade,Name,Gender,Weight,Transfer
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,46.0,N
1,Peking University,Freshman,Changqiang You,Male,70.0,N
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,89.0,N
3,Fudan University,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Fudan University,Sophomore,Gaojuan You,Male,74.0,N


In [10]:
df.query('(Gender == "Male") and (Grade not in ["Freshman","Sophomore"])').head()

,School,Grade,Name,Gender,Weight,Transfer
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,89.0,N
16,Tsinghua University,Junior,Xiaoqiang Qin,Male,68.0,N
17,Tsinghua University,Junior,Peng Wang,Male,65.0,N
18,Tsinghua University,Senior,Xiaofeng Sun,Male,71.0,N
21,Shanghai Jiao Tong University,Senior,Xiaopeng Shen,Male,62.0,NaN


In [21]:
low, high = 70, 80
# df.query('Weight.between (@low, @high)').head()该写法报错
df.query('(Weight>=@low) | (Weight<=@high)').head()

,School,Grade,Name,Gender,Weight,Transfer
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,46.0,N
1,Peking University,Freshman,Changqiang You,Male,70.0,N
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,89.0,N
3,Fudan University,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Fudan University,Sophomore,Gaojuan You,Male,74.0,N


In [23]:
df_sample = pd.DataFrame({'id':list('abcde'), 'value':[1,2,3,4,90]})
df_sample

,id,value
0,a,1
1,b,2
2,c,3
3,d,4
4,e,90


In [27]:
df_sample.sample(3, replace=True, weights=[1,0.5,0.5,0.1,0.1])

,id,value
0,a,1
1,b,2
0,a,1


## 多级索引
### 多级索引及其表的结构
1. 索引的名字获得
   - 行索引名字`df.index.names`
   - 列索引名字`df.columns.names`
2. 索引的值获得
   - 行索引值`df.index.values`
   - 列索引值`df.columns.values`
3. 获得某一层索引`df.index.get_level_values(n)`
### 多级索引中的loc索引器
1. loc和iloc方法可以照搬，只需要把标量的位置替换成对应的元组
2. 传入以下情况需要先对索引进行排序`sort_index`
   - 传入元组列表或单个元组或返回前二者的函数
   - 切片
3. 在多级索引中的元组可以对多层的元素进行交叉组合后索引，同时需要指定loc的列
### IndexSlice对象
1. 引入IndexSlice对象（需要先定义`idx=pd.IndexSlice`）能对每层索引进行切片，允许将切片和布尔列表混合使用
   - `loc[idx[*,*]]`不能进行多层分别切片，前一个\*表示行的选择，后一个\*表示列的选择
   - `loc[idx[*,*], idx[*,*]]`能够分层进行切片，前一个idx指代的是行索引，后一个idx是列索引，但不支持使用函数
### 多级索引的构造
1. `pd.MultiIndex.from_tuples`根据传入由元组组成的列表进行构造
2. `pd.MultiIndex.from_arrays`根据传入列表中，对应层的列表进行构造
3. `pd.MultiIndex.from_product`根据给定多个列表的笛卡尔积进行构造

In [28]:
np.random.seed(0)
multi_index = pd.MultiIndex.from_product([list('ABCD'), df.Gender.unique()], names=('School', 'Gender'))
multi_column = pd.MultiIndex.from_product([['Height', 'Weight'], df.Grade.unique()], names=('Indicator', 'Grade'))
df_multi = pd.DataFrame(np.c_[(np.random.randn(8,4)*5 + 163).tolist(), (np.random.randn(8,4)*5 + 65).tolist()],
                        index = multi_index, columns = multi_column).round(1)
df_multi

Indicator       Height                           Weight                   \
Grade         Freshman Senior Sophomore Junior Freshman Senior Sophomore   
School Gender                                                              
A      Female    171.8  165.0     167.9  174.2     60.6   55.1      63.3   
       Male      172.3  158.1     167.8  162.2     71.2   71.0      63.1   
B      Female    162.5  165.1     163.7  170.3     59.8   57.9      56.5   
       Male      166.8  163.6     165.2  164.7     62.5   62.8      58.7   
C      Female    170.5  162.0     164.6  158.7     56.9   63.9      60.5   
       Male      150.2  166.3     167.3  159.3     62.4   59.1      64.9   
D      Female    174.3  155.7     163.2  162.1     65.3   66.5      61.8   
       Male      170.7  170.3     163.8  164.9     61.6   63.2      60.9   

Indicator             
Grade         Junior  
School Gender         
A      Female   65.8  
       Male     63.5  
B      Female   74.8  
       Male     68.9  
C      Female   66.9  
       Male     67.1  
D      Female   63.2  
       Male     56.4

In [29]:
df_multi.index.names

FrozenList(['School', 'Gender'])

In [30]:
df_multi.columns.names

FrozenList(['Indicator', 'Grade'])

In [31]:
df_multi.index.values

array([('A', 'Female'), ('A', 'Male'), ('B', 'Female'), ('B', 'Male'),
       ('C', 'Female'), ('C', 'Male'), ('D', 'Female'), ('D', 'Male')],
      dtype=object)

In [32]:
df_multi.columns.values

array([('Height', 'Freshman'), ('Height', 'Senior'),
       ('Height', 'Sophomore'), ('Height', 'Junior'),
       ('Weight', 'Freshman'), ('Weight', 'Senior'),
       ('Weight', 'Sophomore'), ('Weight', 'Junior')], dtype=object)

In [33]:
df_multi.index.get_level_values(0)

Index(['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D'], dtype='object', name='School')

In [34]:
df_multi = df.set_index(['School', 'Grade'])
df_multi.head()

Name  Gender  Weight  \
School                        Grade                                       
Shanghai Jiao Tong University Freshman     Gaopeng Yang  Female    46.0   
Peking University             Freshman   Changqiang You    Male    70.0   
Shanghai Jiao Tong University Senior            Mei Sun    Male    89.0   
Fudan University              Sophomore    Xiaojuan Sun  Female    41.0   
                              Sophomore     Gaojuan You    Male    74.0   

                                        Transfer  
School                        Grade               
Shanghai Jiao Tong University Freshman         N  
Peking University             Freshman         N  
Shanghai Jiao Tong University Senior           N  
Fudan University              Sophomore        N  
                              Sophomore        N

In [35]:
# 所有北大和复旦的大二大三学生
res = df_multi.loc[(['Peking University', 'Fudan University'], ['Sophomore', 'Junior']), :]
res.head()

Name  Gender  Weight Transfer
School            Grade                                           
Peking University Sophomore   Changmei Xu  Female    43.0        N
                  Sophomore  Xiaopeng Qin    Male     NaN        N
                  Sophomore        Mei Xu  Female    39.0        N
                  Sophomore   Xiaoli Zhou  Female    55.0        N
                  Sophomore      Peng Han  Female    34.0      NaN

In [36]:
np.random.seed(0)
L1,L2 = ['A','B','C'],['a','b','c']
mul_index1 = pd.MultiIndex.from_product([L1,L2],names=('Upper', 'Lower'))
L3,L4 = ['D','E','F'],['d','e','f']
mul_index2 = pd.MultiIndex.from_product([L3,L4],names=('Big', 'Small'))
df_ex = pd.DataFrame(np.random.randint(-9,10,(9,9)), index=mul_index1, columns=mul_index2)
df_ex

Big          D        E        F      
Small        d  e  f  d  e  f  d  e  f
Upper Lower                           
A     a      3  6 -9 -6 -6 -2  0  9 -5
      b     -3  3 -8 -3 -2  5  8 -4  4
      c     -1  0  7 -4  6  6 -9  9 -6
B     a      8  5 -2 -9 -8  0 -9  1 -6
      b      2  9 -7 -9 -9 -5 -4 -3 -1
      c      8  6 -5  0  1 -8 -8 -2  0
C     a     -6 -3  2  5  9 -9  5 -6  3
      b      1  2 -5 -3 -5  6 -6  3 -5
      c     -1  5  6 -6  6  4  7  8 -4

In [38]:
idx = pd.IndexSlice

In [39]:
df_ex.loc[idx['C':,('D','f'):]]

Big          D  E        F      
Small        f  d  e  f  d  e  f
Upper Lower                     
C     a      2  5  9 -9  5 -6  3
      b     -5 -3 -5  6 -6  3 -5
      c      6 -6  6  4  7  8 -4

In [41]:
df_ex.loc[idx[:'A','b':], idx['E':,'e':]]

Big          E     F   
Small        e  f  e  f
Upper Lower            
A     b     -2  5 -4  4
      c      6  6  9 -6

In [42]:
my_tuple = [('a','cat'), ('a','dog'), ('b','cat'), ('b','dog')]
pd.MultiIndex.from_tuples(my_tuple, names = ['First', 'Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

In [43]:
my_array = [list('aabb'), ['cat','dog']*2]
pd.MultiIndex.from_arrays(my_array, names=['First','Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

In [44]:
my_list1 = ['a','b']
my_list2 = ['cat','dog']
pd.MultiIndex.from_product([my_list1, my_list2], names=['First','Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

## 索引的常用方法
### 索引层的交换和删除
1. 索引的交换（此处不涉及不同方向索引之间的交换）
   - `df.swaplevel`只能交换两个层
   - `df.reorder_level`可以交换任意层
   - 以上两个函数都可以指定交换的是行索引`axis=0`还是列索引`axis=1`
2. 索引的删除
   - `df.droplevel(n|[n1,n2],axis=0)`
### 索引属性的修改
1. `df.rename_axis(index={'修改前':'修改后'}, columns={'修改前':'修改后'})`可以对**索引层的名字**进行修改，传入字典的映射
2. `df.rename(index={'修改前':'修改后'},columns={'修改前':'修改后'})`可以对**索引的值**进行修改，多级索引需要指定修改的层号`level`
3. 传入参数也可以是函数

In [45]:
np.random.seed(0)
L1,L2,L3 = ['A','B'],['a','b'],['alpha','beta']
mul_index1 = pd.MultiIndex.from_product([L1,L2,L3], names=('Upper', 'Lower','Extra'))
L4,L5,L6 = ['C','D'],['c','d'],['cat','dog']
mul_index2 = pd.MultiIndex.from_product([L4,L5,L6], names=('Big', 'Small', 'Other'))
df_ex = pd.DataFrame(np.random.randint(-9,10,(8,8)), index=mul_index1,  columns=mul_index2)
df_ex

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9
            beta   -9  -5  -4  -3  -1   8   6  -5
      b     alpha   0   1  -8  -8  -2   0  -6  -3
            beta    2   5   9  -9   5  -6   3   1

In [46]:
df_ex.swaplevel(0,2,axis=1)

Other             cat dog cat dog cat dog cat dog
Small               c   c   d   d   c   c   d   d
Big                 C   C   C   C   D   D   D   D
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9
            beta   -9  -5  -4  -3  -1   8   6  -5
      b     alpha   0   1  -8  -8  -2   0  -6  -3
            beta    2   5   9  -9   5  -6   3   1

In [47]:
df_ex.reorder_levels([2,0,1], axis=0)

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Extra Upper Lower                                
alpha A     a       3   6  -9  -6  -6  -2   0   9
beta  A     a      -5  -3   3  -8  -3  -2   5   8
alpha A     b      -4   4  -1   0   7  -4   6   6
beta  A     b      -9   9  -6   8   5  -2  -9  -8
alpha B     a       0  -9   1  -6   2   9  -7  -9
beta  B     a      -9  -5  -4  -3  -1   8   6  -5
alpha B     b       0   1  -8  -8  -2   0  -6  -3
beta  B     b       2   5   9  -9   5  -6   3   1

In [52]:
df_ex.rename_axis(index=lambda x:'Changed_row' if x == 'Upper' else x, columns=lambda x: 'Changed_col' if x == 'Other' else x)

Big                       C               D            
Small                     c       d       c       d    
Changed_col             cat dog cat dog cat dog cat dog
Changed_row Lower Extra                                
A           a     alpha   3   6  -9  -6  -6  -2   0   9
                  beta   -5  -3   3  -8  -3  -2   5   8
            b     alpha  -4   4  -1   0   7  -4   6   6
                  beta   -9   9  -6   8   5  -2  -9  -8
B           a     alpha   0  -9   1  -6   2   9  -7  -9
                  beta   -9  -5  -4  -3  -1   8   6  -5
            b     alpha   0   1  -8  -8  -2   0  -6  -3
                  beta    2   5   9  -9   5  -6   3   1

In [53]:
# 对于整个索引的元素替换，可以利用迭代器实现
new_values = iter(list('abcdefgh'))
df_ex.rename(index=lambda x: next(new_values), level=2)

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     a       3   6  -9  -6  -6  -2   0   9
            b      -5  -3   3  -8  -3  -2   5   8
      b     c      -4   4  -1   0   7  -4   6   6
            d      -9   9  -6   8   5  -2  -9  -8
B     a     e       0  -9   1  -6   2   9  -7  -9
            f      -9  -5  -4  -3  -1   8   6  -5
      b     g       0   1  -8  -8  -2   0  -6  -3
            h       2   5   9  -9   5  -6   3   1